
```
███╗   ███╗██╗███╗   ██╗███████╗ ██████╗ ██████╗ ██╗      █████╗ ██████╗
████╗ ████║██║████╗  ██║██╔════╝██╔════╝██╔═══██╗██║     ██╔══██╗██╔══██╗
██╔████╔██║██║██╔██╗ ██║█████╗  ██║     ██║   ██║██║     ███████║██████╔╝
██║╚██╔╝██║██║██║╚██╗██║██╔══╝  ██║     ██║   ██║██║     ██╔══██║██╔══██╗
██║ ╚═╝ ██║██║██║ ╚████║███████╗╚██████╗╚██████╔╝███████╗██║  ██║██████╔╝
╚═╝     ╚═╝╚═╝╚═╝  ╚═══╝╚══════╝ ╚═════╝ ╚═════╝ ╚══════╝╚═╝  ╚═╝╚═════╝
```
**Run a Minecraft Server on Google Colab!**

---

The script below will run your server. You'll have to create a server first to be able to use it - don't worry, the scripts below will do the majority of the work for you. You might also want to change the default region to your region, check below.

In [11]:
import os
import json
import requests
from google.colab import drive
from datetime import datetime
import subprocess

# ================= CONFIGURATION =================
DRIVE_FOLDER = "/content/drive/MyDrive/Minecraft-Server-1.21.5"  # Your Drive folder
MEMORY_ALLOCATION = "-Xmx6G -Xms6G"  # 6GB RAM
DISCORD_WEBHOOK = ""  # Leave empty or add your webhook URL
AUTO_BACKUP = True
TUNNEL_SERVICE = "argo"  # "argo", "ngrok", or "playit"

# ================= IMPROVED SETUP =================
def setup_environment():
    print("🚀 Setting up Minecraft 1.21.5 Server...")

    # Mount Google Drive with force remount
    drive.mount('/content/drive', force_remount=True)
    os.makedirs(DRIVE_FOLDER, exist_ok=True)
    os.chdir(DRIVE_FOLDER)
    print(f"📂 Working directory: {os.getcwd()}")

    # Install Java 21 with better error handling
    print("\n🔧 Installing Java 21...")
    java_install = subprocess.run(
        "sudo apt update -qq && sudo apt install -qqy openjdk-21-jdk",
        shell=True,
        capture_output=True,
        text=True
    )

    if java_install.returncode == 0:
        print("✅ Java 21 installed successfully")
    else:
        print(f"❌ Java installation failed: {java_install.stderr}")
        return False

    # Verify Java version properly
    java_ver = subprocess.run(
        "java -version 2>&1 | head -n 1 | awk -F'\"' '{print $2}'",
        shell=True,
        capture_output=True,
        text=True
    ).stdout.strip()

    print(f"ℹ️ Detected Java version: {java_ver}")
    return True

# ================= ENHANCED SERVER DOWNLOAD =================
def download_server():
    print("\n⬇️ Downloading Minecraft 1.21.5 server...")

    try:
        # Get version manifest with timeout
        version_manifest = requests.get(
            "https://launchermeta.mojang.com/mc/game/version_manifest.json",
            timeout=10
        ).json()

        # Find 1.21.5 version
        version_data = next(
            v for v in version_manifest["versions"]
            if v["id"] == "1.21.5"
        )

        # Download server.jar with progress
        server_url = requests.get(version_data["url"]).json()["downloads"]["server"]["url"]
        download = subprocess.run(
            f"wget -q --show-progress -O server.jar {server_url}",
            shell=True
        )

        if download.returncode == 0:
            print("✅ server.jar downloaded successfully")

            # Create required files
            with open("eula.txt", "w") as f:
                f.write("eula=true\n")
            print("✅ EULA accepted automatically")

            return True
        else:
            print("❌ Failed to download server.jar")
            return False

    except Exception as e:
        print(f"❌ Error during setup: {str(e)}")
        return False

# ================= ROBUST BACKUP SYSTEM =================
def backup_world():
    if not AUTO_BACKUP:
        return

    try:
        backup_dir = os.path.join(DRIVE_FOLDER, "backups")
        os.makedirs(backup_dir, exist_ok=True)
        timestamp = datetime.now().strftime("%Y%m%d-%H%M%S")
        backup_file = f"world-backup-{timestamp}.zip"

        print(f"\n💾 Creating backup: {backup_file}")

        # Only backup if world exists
        if os.path.exists("world"):
            result = subprocess.run(
                f"zip -qr {os.path.join(backup_dir, backup_file)} world/",
                shell=True
            )
            if result.returncode == 0:
                print("✅ Backup completed successfully")
                return True
            else:
                print("❌ Backup failed")
                return False
        else:
            print("⚠️ No world directory found to backup")
            return False
    except Exception as e:
        print(f"❌ Backup error: {str(e)}")
        return False

# ================= IMPROVED DISCORD INTEGRATION =================
def send_discord_message(message):
    if not DISCORD_WEBHOOK or not DISCORD_WEBHOOK.startswith("https://"):
        return False

    try:
        payload = {
            "content": f"🟢 **Minecraft Server**: {message}",
            "username": "Minecraft Server Bot"
        }
        response = requests.post(
            DISCORD_WEBHOOK,
            json=payload,
            timeout=5
        )
        return response.status_code == 200
    except Exception as e:
        print(f"❌ Discord notification failed: {str(e)}")
        return False

# ================= SERVER MANAGEMENT =================
def start_server():
    print("\n🔄 Starting server...")
    send_discord_message("Server starting up...")

    # Start tunnel
    tunnel_process = None
    if TUNNEL_SERVICE == "argo":
        print("🌐 Setting up Cloudflare Tunnel...")
        tunnel_process = subprocess.Popen(
            "./cloudflared-linux-amd64 tunnel --url tcp://localhost:25565",
            shell=True
        )

    # Start server
    try:
        server_process = subprocess.Popen(
            f"java {MEMORY_ALLOCATION} -jar server.jar nogui",
            shell=True
        )
        server_process.wait()

    except KeyboardInterrupt:
        print("\n🛑 Received shutdown signal...")
    finally:
        print("\n🔴 Server stopping...")
        if tunnel_process:
            tunnel_process.terminate()

        if backup_world():
            send_discord_message("Server shutdown complete. World backup saved.")
        else:
            send_discord_message("Server shutdown complete (no backup created).")

        print("✅ Clean shutdown completed")

# ================= MAIN EXECUTION =================
if __name__ == "__main__":
    if not setup_environment():
        print("❌ Setup failed, cannot continue")
        exit(1)

    if not os.path.exists("server.jar") and not download_server():
        print("❌ Server download failed, cannot continue")
        exit(1)

    # Download cloudflared if using Argo
    if TUNNEL_SERVICE == "argo" and not os.path.exists("cloudflared-linux-amd64"):
        print("⬇️ Downloading cloudflared...")
        subprocess.run(
            "wget -q https://github.com/cloudflare/cloudflared/releases/latest/download/cloudflared-linux-amd64",
            shell=True
        )
        subprocess.run("chmod +x cloudflared-linux-amd64", shell=True)
    elif TUNNEL_SERVICE == "ngrok":
      !pip install -q pyngrok
      from pyngrok import ngrok

      # Authenticate (replace with your token)
      ngrok.set_auth_token("YOUR_AUTHTOKEN")

      # Start tunnel
      mc_tunnel = ngrok.connect(25565, "tcp")
      public_ip = mc_tunnel.public_url.replace("tcp://", "")
      print(f"\n✅ Ngrok Tunnel Active! Connect to: {public_ip}")

    # Send to Discord (if webhook is set)
    if DISCORD_WEBHOOK:
        send_discord_message(f"Server is live at `{public_ip}`")

    start_server()

🚀 Setting up Minecraft 1.21.5 Server...
Mounted at /content/drive
📂 Working directory: /content/drive/MyDrive/Minecraft-Server-1.21.5

🔧 Installing Java 21...
✅ Java version: 2

⬇️ Downloading Minecraft 1.21.5 server...
✅ Downloaded server.jar
✅ Accepted EULA

🔄 Starting server with tunnel...
❌ Discord notification failed: Invalid URL 'YOUR_DISCORD_WEBHOOK_URL': No scheme supplied. Perhaps you meant https://YOUR_DISCORD_WEBHOOK_URL?
2025-06-11T18:09:54Z INF Thank you for trying Cloudflare Tunnel. Doing so, without a Cloudflare account, is a quick way to experiment and try it out. However, be aware that these account-less Tunnels have no uptime guarantee, are subject to the Cloudflare Online Services Terms of Use (https://www.cloudflare.com/website-terms/), and Cloudflare reserves the right to investigate your use of Tunnels for violations of such terms. If you intend to use Tunnels in production you should use a pre-created named tunnel by following: https://developers.cloudflare.com/c

# Make-a-Server

The code below will download a server for you and accept the EULA. After running these scripts, your server will be ready to run.

**Download the Minecraft server**

The code below will download Paper, a high-performance fork of the Vanilla server.
Other server platforms can be used by placing the server.jar in the Drive folder manually.

In [3]:
from google.colab import drive
import requests
import json

# Mount Google Drive
drive.mount('/content/drive')

# Create Minecraft server directory
!mkdir -p "/content/drive/My Drive/Minecraft-server"
%cd "/content/drive/My Drive/Minecraft-server"

# Fetch latest Minecraft version info
print("Fetching latest Minecraft version...")
version_manifest = requests.get("https://launchermeta.mojang.com/mc/game/version_manifest.json").json()
latest_version = version_manifest['latest']['release']

# Find the server download URL for the latest version
print(f"Getting download URL for Minecraft {latest_version}...")
for version in version_manifest['versions']:
    if version['id'] == latest_version:
        version_data = requests.get(version['url']).json()
        server_url = version_data['downloads']['server']['url']
        break

# Download the server
print(f"Downloading Minecraft {latest_version} server...")
r = requests.get(server_url)

if r.status_code == 200:
    with open('server.jar', 'wb') as f:
        f.write(r.content)
    print("Download complete!")
else:
    print(f"Error {r.status_code}! Failed to download the server.")
    exit()

# Create EULA agreement
print("Creating eula.txt...")
with open('eula.txt', 'w') as f:
    f.write("eula=true\n")

# Save server info
server_info = {
    "version": latest_version,
    "type": "vanilla",
    "server_jar": "server.jar"
}
with open('server_info.json', 'w') as f:
    json.dump(server_info, f)

print("\nSetup complete! Server files are in:")
print("/content/drive/My Drive/Minecraft-server/")
print(f"\nMinecraft {latest_version} (Vanilla) server.jar is ready.")

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
/content/drive/My Drive/Minecraft-server
Fetching latest Minecraft version...
Getting download URL for Minecraft 1.21.5...
Download complete!
Creating eula.txt...

Setup complete! Server files are in:
/content/drive/My Drive/Minecraft-server/

Minecraft 1.21.5 (Vanilla) server.jar is ready.


**Automatically accept the EULA**

In [4]:
# Please read the file stored in your server folder before running this command.
# Also, go to https://www.minecraft.net/en-us/eula to read Minecraft's EULA.

# Make sure Drive is mounted
from google.colab import drive
drive.mount('/content/drive')

%cd "/content/drive/My Drive/Minecraft-server"
!echo "eula=true" >> eula.txt

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
/content/drive/My Drive/Minecraft-server


# Debug
SSH access to host OS - Thanks to [colab-ssh](https://github.com/WassimBenzarti/colab-ssh).

<p style="color:red;">YOU MIGHT GET BANNED</p>

In [ ]:
#@title Colab-ssh tunnel
#@markdown Execute this cell to open the ssh tunnel. Check [colab-ssh documentation](https://github.com/WassimBenzarti/colab-ssh) for more details.

# Install colab_ssh on google colab
!pip install colab_ssh --upgrade

from colab_ssh import launch_ssh_cloudflared, init_git_cloudflared
ssh_tunnel_password = "<PUT_YOUR_PASSWORD_HERE>" #@param {type: "string"}
launch_ssh_cloudflared(password=ssh_tunnel_password)

In [ ]:
#Get public address (ngrok)
! curl -s http://localhost:4040/api/tunnels | python3 -c \
    "import sys, json; print(json.load(sys.stdin)['tunnels'][0]['public_url'])"

In [5]:
## For inspecting the minecraft server directory ##
%cd "/content/drive/My Drive/Minecraft-server"
!ls


/content/drive/My Drive/Minecraft-server
eula.txt  server_info.json  server.jar
